<a href="https://colab.research.google.com/github/ritallopes/OFP-ES/blob/preprocessing/OFP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
import pandas as pd
import matplotlib.pyplot as plt


In [107]:
#definindo estados do nordeste e suas siglas referentes
etiquetas = ['AL',
    'BA-1',
    'BA-2',
    'CE',
    'MA',
    'PB',
    'PE',
    'PI',
    'RN',
    'SE']
etiquetas

['AL', 'BA-1', 'BA-2', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE']

1.0.  importing and knowing dataset



In [106]:
data_br =pd.DataFrame()
for sigla in etiquetas:
    locals()['covid_%s'%sigla.lower()] = pd.read_csv('https://s3-sa-east-1.amazonaws.com/ckan.saude.gov.br/dados-{}.csv'.format(sigla.lower()), encoding='latin-1', sep=';')
    data_br = pd.concat([data_br ,(locals()['covid_%s'%sigla.lower()])], axis=0, ignore_index=True)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (15,17,21,23) have mixed types.Specify dtype option on import or set low_memory=False.

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (15,21) have mixed types.Specify dtype option on import or set low_memory=False.

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (15,21,26) have mixed types.Specify dtype option on import or set low_memory=False.

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (15,21,23,26) have mixed types.Specify dtype option on import or set low_memory=False.

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2

In [108]:
data_br

,ÿid,dataNotificacao,dataInicioSintomas,dataNascimento,sintomas,profissionalSaude,cbo,condicoes,estadoTeste,dataTeste,tipoTeste,resultadoTeste,paisOrigem,sexo,estado,estadoIBGE,municipio,municipioIBGE,origem,cnes,estadoNotificacao,estadoNotificacaoIBGE,municipioNotificacao,municipioNotificacaoIBGE,excluido,validado,idade,dataEncerramento,evolucaoCaso,classificacaoFinal
0,6DIaF7l5DB,2020-05-26T00:29:06.577Z,2020-05-10T03:00:00.000Z,1985-01-02T03:00:00.000Z,"Febre, Tosse, Outros",Não,NaN,NaN,Coletado,2020-05-25T03:00:00.000Z,NaN,NaN,undefined,Feminino,ALAGOAS,27,Joaquim Gomes,2703809,undefined,undefined,ALAGOAS,27,Joaquim Gomes,2703809,undefined,undefined,35,NaN,NaN,NaN
1,TgsKoaU6YG,2020-04-20T03:00:00.000Z,2020-04-12T03:00:00.000Z,1976-08-27T03:00:00.000Z,"Dispneia, Febre",Não,NaN,NaN,Concluído,2020-04-19T03:00:00.000Z,RT-PCR,Positivo,undefined,Masculino,ALAGOAS,27,Maceió,2704302,undefined,undefined,ALAGOAS,27,Maceió,2704302,undefined,undefined,43,2020-05-20T03:00:00.000Z,Cura,Confirmado Laboratorial
2,7zp7ZEqRcx,2020-05-15T02:48:06.361Z,2020-04-30T03:00:00.000Z,1989-12-09T02:00:00.000Z,"Febre, Tosse, Dor de Garganta, Dispneia",Sim,2232 - CIRURGIÃO-DENTISTA,NaN,Concluído,2020-05-11T03:00:00.000Z,TESTE RÁPIDO - ANTICORPO,Positivo,undefined,Feminino,ALAGOAS,27,São José da Laje,2708303,undefined,2722356,ALAGOAS,27,São José da Laje,2708303,undefined,undefined,30,2020-05-20T03:00:00.000Z,Em tratamento domiciliar,Confirmado Laboratorial
3,0fVCqcORJ1,2020-04-21T10:44:54.764Z,2020-04-16T03:00:00.000Z,1975-04-02T03:00:00.000Z,"Dor de Garganta, Febre, Outros",Não,NaN,NaN,Concluído,2020-04-21T03:00:00.000Z,RT-PCR,Positivo,undefined,Feminino,ALAGOAS,27,Maceió,2704302,undefined,980016281450332,ALAGOAS,27,Maceió,2704302,undefined,undefined,45,2020-04-24T03:00:00.000Z,Cura,Confirmado Laboratorial
4,Go6HUkB6MD,2020-04-21T07:07:11.482Z,2020-04-16T03:00:00.000Z,1958-04-27T03:00:00.000Z,"Febre, Outros",Não,NaN,NaN,Concluído,2020-04-21T03:00:00.000Z,RT-PCR,Positivo,undefined,Feminino,ALAGOAS,27,Maceió,2704302,undefined,undefined,ALAGOAS,27,Maceió,2704302,undefined,undefined,62,2020-04-24T03:00:00.000Z,Cura,Confirmado Laboratorial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4926866,CCp86HDc71,2020-10-16T03:00:00.000Z,2020-10-16T03:00:00.000Z,1994-12-30T02:00:00.000Z,Assintomático,Não,NaN,NaN,Coletado,2020-10-16T03:00:00.000Z,RT-PCR,NaN,undefined,Feminino,SERGIPE,28,Poço Verde,2805505,undefined,undefined,SERGIPE,28,Poço Verde,2805505,undefined,undefined,25,NaN,NaN,NaN
4926867,EFxdWL3Czl,2020-10-17T03:00:42.595Z,2020-10-13T03:00:00.000Z,2001-12-17T02:00:00.000Z,"Tosse, Dispneia",Não,NaN,NaN,Coletado,2020-10-16T03:00:00.000Z,RT-PCR,NaN,undefined,Feminino,SERGIPE,28,Pedrinhas,2805109,undefined,6901743,SERGIPE,28,Estância,2802106,undefined,undefined,18,NaN,NaN,NaN
4926868,kIBjlDzZMV,2020-10-17T03:00:05.142Z,2020-10-13T03:00:00.000Z,1964-10-22T03:00:00.000Z,"Tosse, Dispneia",Não,NaN,NaN,Coletado,2020-10-16T03:00:00.000Z,RT-PCR,NaN,undefined,Masculino,SERGIPE,28,Indiaroba,2802809,undefined,6901743,SERGIPE,28,Estância,2802106,undefined,undefined,55,NaN,NaN,NaN
4926869,iUAAlwVDR9,2020-10-17T03:00:06.995Z,2020-10-13T03:00:00.000Z,1970-11-08T03:00:00.000Z,"Tosse, Dispneia",Não,NaN,NaN,Coletado,2020-10-16T03:00:00.000Z,RT-PCR,NaN,undefined,Masculino,SERGIPE,28,Estância,2802106,undefined,6901743,SERGIPE,28,Estância,2802106,undefined,undefined,49,NaN,NaN,NaN


In [89]:
covid_ce.shape

(818848, 30)

In [87]:
covid_ba.shape

(1276388, 30)

In [61]:
covid_rn.tail()


,ÿid,dataNotificacao,dataInicioSintomas,dataNascimento,sintomas,profissionalSaude,cbo,condicoes,estadoTeste,dataTeste,tipoTeste,resultadoTeste,paisOrigem,sexo,estado,estadoIBGE,municipio,municipioIBGE,origem,cnes,estadoNotificacao,estadoNotificacaoIBGE,municipioNotificacao,municipioNotificacaoIBGE,excluido,validado,idade,dataEncerramento,evolucaoCaso,classificacaoFinal
404393,h9wdVZcMUY,2020-10-16T03:00:00.538Z,2020-10-11T03:00:00.000Z,1983-09-08T03:00:00.000Z,"Tosse, Dor de Garganta",Não,NaN,Doenças cardíacas crônicas,Solicitado,NaN,RT-PCR,NaN,undefined,Masculino,RIO GRANDE DO NORTE,24,Jundiá,2406155,undefined,2375168,RIO GRANDE DO NORTE,24,Jundiá,2406155,undefined,undefined,37,NaN,NaN,NaN
404394,0uX955ZAyP,2020-10-16T03:00:07.539Z,2020-10-16T03:00:00.000Z,1989-07-17T03:00:00.000Z,Assintomático,Não,NaN,NaN,Concluído,2020-10-16T03:00:00.000Z,TESTE RÁPIDO - ANTICORPO,Negativo,undefined,Masculino,RIO GRANDE DO NORTE,24,Cerro Corá,2402709,undefined,undefined,MINAS GERAIS,31,Araguari,3103504,undefined,undefined,31,NaN,NaN,NaN
404395,SwZYCNbXIF,2020-10-16T03:00:57.083Z,2020-10-14T03:00:00.000Z,1992-06-21T03:00:00.000Z,"Tosse, Dispneia, Coriza, Outros",Não,NaN,NaN,Exame Não Solicitado,NaN,NaN,NaN,undefined,Masculino,RIO GRANDE DO NORTE,24,Natal,2408102,undefined,2408627,RIO GRANDE DO NORTE,24,Natal,2408102,undefined,undefined,28,NaN,NaN,NaN
404396,OMspYNNZO3,2020-10-16T03:00:32.971Z,2020-10-14T03:00:00.000Z,2006-01-01T02:00:00.000Z,"Febre, Tosse, Dor de Cabeça",Não,NaN,NaN,Coletado,2020-10-16T03:00:00.000Z,TESTE RÁPIDO - ANTÍGENO,NaN,undefined,Masculino,RIO GRANDE DO NORTE,24,Pedro Velho,2409803,undefined,980016287354521,RIO GRANDE DO NORTE,24,Pedro Velho,2409803,undefined,undefined,14,NaN,NaN,NaN
404397,CmPI5jfnmZ,2020-10-16T03:00:30.830Z,2020-10-14T03:00:00.000Z,2006-04-26T03:00:00.000Z,Dor de Cabeça,Não,NaN,NaN,Solicitado,NaN,RT-PCR,NaN,undefined,Feminino,RIO GRANDE DO NORTE,24,Pedro Velho,2409803,undefined,980016287354521,RIO GRANDE DO NORTE,24,Pedro Velho,2409803,undefined,undefined,14,NaN,NaN,NaN


In [62]:
covid_rn.condicoes.unique()
#doenças pre-existentes
covid_rn.loc[covid_rn.evolucaoCaso=="Óbito","condicoes"].unique()
#doencas cronicas cardiacas e respiratórias em quase todas as mortes documentadas
covid_rn.validado.unique()

array(['undefined'], dtype=object)

In [63]:
 covid_rn.columns

Index(['ÿid', 'dataNotificacao', 'dataInicioSintomas', 'dataNascimento',
       'sintomas', 'profissionalSaude', 'cbo', 'condicoes', 'estadoTeste',
       'dataTeste', 'tipoTeste', 'resultadoTeste', 'paisOrigem', 'sexo',
       'estado', 'estadoIBGE', 'municipio', 'municipioIBGE', 'origem', 'cnes',
       'estadoNotificacao', 'estadoNotificacaoIBGE', 'municipioNotificacao',
       'municipioNotificacaoIBGE', 'excluido', 'validado', 'idade',
       'dataEncerramento', 'evolucaoCaso', 'classificacaoFinal'],
      dtype='object')

In [84]:
covid_rn.estadoIBGE

0         24
1         24
2         24
3         24
4         24
          ..
404393    24
404394    24
404395    24
404396    24
404397    24
Name: estadoIBGE, Length: 404398, dtype: object

In [64]:
covid_ba.columns

Index(['ÿid', 'dataNotificacao', 'dataInicioSintomas', 'dataNascimento',
       'sintomas', 'profissionalSaude', 'cbo', 'condicoes', 'estadoTeste',
       'dataTeste', 'tipoTeste', 'resultadoTeste', 'paisOrigem', 'sexo',
       'estado', 'estadoIBGE', 'municipio', 'municipioIBGE', 'origem', 'cnes',
       'estadoNotificacao', 'estadoNotificacaoIBGE', 'municipioNotificacao',
       'municipioNotificacaoIBGE', 'excluido', 'validado', 'idade',
       'dataEncerramento', 'evolucaoCaso', 'classificacaoFinal'],
      dtype='object')

In [65]:
covid_ce.columns

Index(['ÿid', 'dataNotificacao', 'dataInicioSintomas', 'dataNascimento',
       'sintomas', 'profissionalSaude', 'cbo', 'condicoes', 'estadoTeste',
       'dataTeste', 'tipoTeste', 'resultadoTeste', 'paisOrigem', 'sexo',
       'estado', 'estadoIBGE', 'municipio', 'municipioIBGE', 'origem', 'cnes',
       'estadoNotificacao', 'estadoNotificacaoIBGE', 'municipioNotificacao',
       'municipioNotificacaoIBGE', 'excluido', 'validado', 'idade',
       'dataEncerramento', 'evolucaoCaso', 'classificacaoFinal'],
      dtype='object')

In [72]:
#Os profissionais que constam no dataset
covid_rn.loc[covid_rn.profissionalSaude=="Sim","cbo"].unique()
#não foram somente profissionais diretamente ligados à saúde que estão relacionados no dataset

array(['3222 - TÉCNICO OU AUXILIAR EM ENFERMAGEM', '2234 - FARMACÊUTICO',
       '225 - MÉDICO', '4221 - RECEPCIONISTA',
       '515140 - AGENTE DE COMBATE A ENDEMIAS', '2237 - NUTRISIONISTA',
       '515105 - AGENTE COMUNITÁRIO DE SAÚDE', '2235 - ENFERMEIRO',
       '239 - OUTROS PROFISSIONAIS DE ENSINO',
       '2011 - PROFISSIONAL DA BIOTECNOLOGIA',
       '3242 - TÉCNICO DE LABORATÓRIO DE SAÚDE OU BANCOS DE SANGUE',
       '2232 - CIRURGIÃO-DENTISTA',
       '3224\xa0- TÉCNICO OU AUXILIAR ODONTOLOGIA/SAÚDE BUCAL',
       '352210 - AGENTE DE SAÚDE PÚBLICA',
       '782320 - CONDUTOR DE AMBULÂNCIA',
       '2131 - FÍSICO ATUANDO NA ÁREA DA SAÚDE', '2236 - FISIOTERAPEUTA',
       '1312 - GESTORES E ESPECIALISTAS DE OPERAÇÕES EM EMPRESAS, SECRETARIAS E UNIDADES DE SERVIÇOS DE SAÚDE',
       '3251\xa0- TÉCNICO EM FARMÁCIA E MANIPULAÇÃO FARMACÊUTICA',
       '2212 - BIOMÉDICO',
       '4222 - TELEFONISTA, TELEOPERADOR, MONITOR DE TELEATENDIMENTO OU OPERADOR DE RÁDIO-CHAMADA',
       '213

In [ ]:
#Sintomas de quem teve morte confirmada e documentada
covid_rn.loc[covid_rn.evolucaoCaso=="Óbito","sintomas"].unique()
#nenhum assintomatico teve morte documentada

2.0. CLEANING DATASETS

In [79]:
data_br.estado.unique()

array(['RIO GRANDE DO NORTE', 'CEARÁ', 'DISTRITO FEDERAL', 'PARAÍBA',
       'PIAUÍ', 'PERNAMBUCO', nan, 'BAHIA', 'RIO DE JANEIRO', 'SÃO PAULO',
       'SANTA CATARINA', 'PARÁ', 'MINAS GERAIS', 'SERGIPE', 'RONDÔNIA',
       'MARANHÃO', 'PARANÁ', 'MATO GROSSO', 'ALAGOAS', 'AMAZONAS',
       'GOIÁS', 'ACRE', 'MATO GROSSO DO SUL', 'RIO GRANDE DO SUL',
       'TOCANTINS', 'RORAIMA', 'ESPÍRITO SANTO', 'AMAPÁ'], dtype=object)

In [81]:
data_rn = covid_rn.iloc[:,[0,2,5,11,13,14,26,27, 28]].copy() #choose columns
data_rn.columns = ['id', 'inicioSintomas', 'profissional', 'resultado', 'sexo', 'estado', 'idade', 'encerramento', 'evolucao']

In [82]:
data_rn.index = data_rn.id
data_rn.drop(labels="id",axis=1,inplace=True) 
data_rn.head()
data_rn.estado.unique()

array(['RIO GRANDE DO NORTE', 'CEARÁ', 'DISTRITO FEDERAL', 'PARAÍBA',
       'PIAUÍ', 'PERNAMBUCO', nan, 'BAHIA', 'RIO DE JANEIRO', 'SÃO PAULO',
       'SANTA CATARINA', 'PARÁ', 'MINAS GERAIS', 'SERGIPE', 'RONDÔNIA',
       'MARANHÃO', 'PARANÁ', 'MATO GROSSO', 'ALAGOAS', 'AMAZONAS',
       'GOIÁS', 'ACRE', 'MATO GROSSO DO SUL', 'RIO GRANDE DO SUL',
       'TOCANTINS', 'RORAIMA', 'ESPÍRITO SANTO', 'AMAPÁ'], dtype=object)

In [70]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import numpy as np

# Initialize figure with 4 3D subplots
fig = make_subplots(
    rows=3, cols=3,
    specs=[[{'type': 'surface'}, {'type': 'surface'}, {'type': 'surface'}],
           [{'type': 'surface'}, {'type': 'surface'}, {'type': 'surface'}],
           [{'type': 'surface'}, {'type': 'surface'}, {'type': 'surface'}]])

# Generate data
x = np.linspace(-5, 60, 10)
y = np.linspace(-5, 60, 10)
xGrid, yGrid = np.meshgrid(y, x)
z = xGrid ** 3 + yGrid ** 3

# adding surfaces to subplots.
fig.add_trace(
    go.Surface(x=x, y=y, z=z, colorscale='Viridis', showscale=False),
    row=1, col=1)

fig.add_trace(
    go.Surface(x=x, y=y, z=z, colorscale='RdBu', showscale=False),
    row=1, col=2)

fig.add_trace(
    go.Surface(x=x, y=y, z=z, colorscale='RdBu', showscale=False),
    row=1, col=3)

fig.add_trace(
    go.Surface(x=x, y=y, z=z, colorscale='YlOrRd', showscale=False),
    row=2, col=1)

fig.add_trace(
    go.Surface(x=x, y=y, z=z, colorscale='YlGnBu', showscale=False),
    row=2, col=2)

fig.add_trace(
    go.Surface(x=x, y=y, z=z, colorscale='YlGnBu', showscale=False),
    row=2, col=3)

fig.add_trace(
    go.Surface(x=x, y=y, z=z, colorscale='YlOrRd', showscale=False),
    row=3, col=1)

fig.add_trace(
    go.Surface(x=x, y=y, z=z, colorscale='YlGnBu', showscale=False),
    row=3, col=2)

fig.add_trace(
    go.Surface(x=x, y=y, z=z, colorscale='YlGnBu', showscale=False),
    row=3, col=3)


fig.update_layout(
    title_text='3D subplots with different colorscales',
    height=800,
    width=800
)

fig.show()


In [ ]:
rn_prof = covid_rn.loc[covid_rn.profissionalSaude == "Sim"].copy()

In [ ]:
covid_rn.condicoes.unique()

In [ ]:
rn_prof.loc[rn_prof.evolucaoCaso == "Óbito"]

In [ ]:
rn_prof